# Downloading Data

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np

In [4]:
list_stock=["AAPL", "MSFT", "AMZN", "NVDA", "TSLA", "META"]
data = yf.download(list_stock, start='2015-01-01', end='2022-12-31')['Adj Close']
data

[*********************100%%**********************]  6 of 6 completed


,AAPL,AMZN,META,MSFT,NVDA,TSLA
Date,,,,,,
2015-01-02,24.466450,15.426000,78.449997,40.452740,4.833230,14.620667
2015-01-05,23.777191,15.109500,77.190002,40.080734,4.751596,14.006000
2015-01-06,23.779425,14.764500,76.150002,39.492458,4.607536,14.085333
2015-01-07,24.112871,14.921000,76.150002,39.994228,4.595531,14.063333
2015-01-08,25.039337,15.023000,78.180000,41.170788,4.768403,14.041333
...,...,...,...,...,...,...
2022-12-23,131.127060,85.250000,118.040001,236.631805,152.005920,123.150002
2022-12-27,129.307236,83.040001,116.879997,234.877380,141.159790,109.099998
2022-12-28,125.339417,81.820000,115.620003,232.468719,140.310074,112.709999


In [5]:
monthly_prices = data.resample('M').last()  # Use 'last' to get the last price of each month
monthly_returns = monthly_prices.pct_change()

# Assuming risk-free rate is 0, excess returns are the same as raw returns
excess_returns = monthly_returns

excess_returns

,AAPL,AMZN,META,MSFT,NVDA,TSLA
Date,,,,,,
2015-01-31,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-28,0.100777,0.072293,0.040311,0.093120,0.153382,-0.001277
2015-03-31,-0.031372,-0.021202,0.041155,-0.072748,-0.051224,-0.071653
2015-04-30,0.005786,0.133513,-0.041961,0.196261,0.060678,0.197489
2015-05-31,0.045339,0.017663,0.005332,-0.030334,0.001461,0.109489
...,...,...,...,...,...,...
2022-08-31,-0.031208,-0.060615,0.024073,-0.066663,-0.168970,-0.072489
2022-09-30,-0.120977,-0.108622,-0.167250,-0.109267,-0.195534,-0.037589
2022-10-31,0.109551,-0.093451,-0.313384,-0.003306,0.111871,-0.142168


# Question 1:
- Based on the sample data, compute the Markowitz portfolio weights.

# Index Weighting:
- Apple : 7.10%
- Microsoft: 6.51%
- Amazon: 3.24%
- NVIDIA: 2.84%
- Tesla : 1.87%
- Meta : 1.84%

In [6]:
weight=np.array([7.10, 6.51, 3.24, 2.84, 1.87, 1.84])
weight/100

array([0.071 , 0.0651, 0.0324, 0.0284, 0.0187, 0.0184])

In [7]:
ones=np.ones((10,1))
cov_matrix=excess_returns.cov()
cov_matrix_inverse=np.linalg.inv(cov_matrix)

In [8]:
cov_matrix_inverse*(excess_returns["AAPL"].mean())

array([[ 6.10761817e+00, -3.74981363e-01, -4.46334217e-01,
        -2.02369098e+00, -1.04274374e+00, -8.19831810e-01],
       [-3.74981363e-01,  5.15307170e+00, -8.54774987e-01,
        -3.12646700e+00, -4.92595776e-01, -3.29388860e-01],
       [-4.46334217e-01, -8.54774987e-01,  3.10267453e+00,
        -9.26927777e-01, -2.93126733e-02,  1.17497863e-02],
       [-2.02369098e+00, -3.12646700e+00, -9.26927777e-01,
         1.24418327e+01, -1.33901432e+00, -2.36227553e-01],
       [-1.04274374e+00, -4.92595776e-01, -2.93126733e-02,
        -1.33901432e+00,  2.03701367e+00,  9.72282626e-02],
       [-8.19831810e-01, -3.29388860e-01,  1.17497863e-02,
        -2.36227553e-01,  9.72282626e-02,  9.02971132e-01]])

In [9]:
# As in the course, we take γmkt = 3.0271189.

gamma=3.0271189
weight_mvu=1/gamma*cov_matrix_inverse@(excess_returns.mean())
data_weights=pd.DataFrame(index=list_stock)
data_weights['Mean Excess return %']=list(excess_returns.mean()*100)
data_weights['Weights Market %']=weight
data_weights['Weights MVU %']=list(weight_mvu*100)
data_weights

,Mean Excess return %,Weights Market %,Weights MVU %
AAPL,2.041488,7.10,-18.505199
MSFT,2.057845,6.51,-23.694878
AMZN,0.951350,3.24,-30.376338
NVDA,2.231979,2.84,150.187995
TSLA,4.588607,1.87,57.565217
META,3.798792,1.84,16.323382


# Question 2:
- Then, using the market-capitalization weights, obtain the CAPM-implied
expected returns.

In [10]:
gamma*cov_matrix@(weight/100)

AAPL    0.003777
AMZN    0.004072
META    0.003131
MSFT    0.002668
NVDA    0.004984
TSLA    0.006295
dtype: float64

In [11]:
data_weights['Implied CAPM excess returns %']=list((gamma*cov_matrix@(weight/100))*100)
data_weights

,Mean Excess return %,Weights Market %,Weights MVU %,Implied CAPM excess returns %
AAPL,2.041488,7.10,-18.505199,0.377696
MSFT,2.057845,6.51,-23.694878,0.407228
AMZN,0.951350,3.24,-30.376338,0.313083
NVDA,2.231979,2.84,150.187995,0.266783
TSLA,4.588607,1.87,57.565217,0.498379
META,3.798792,1.84,16.323382,0.629454


# Question 3:
- Then, specify the pick matrix P and the view vector q that captures the following views for each of the assets:
    - AAPL: its absolute excess return is expected to be 10% per year.
    - MSFT: its absolute excess return is expected to be 5% per year.
    - AMZN: no views
    - NVDA will outperform TSLA by 2% per year.
    - TSLA will underperform META by 1% per year.
- Finally, explain your choice for the matrix Ω, which captures the uncertainty about these views.

In [12]:
P=np.array([
    [1, 0, 0, 0, 0, 0],  # AAPL view
    [0, 1, 0, 0, 0, 0],  # MSFT view
    [0, 0, 0, 0, 0, 0],  # AMZN view (no view)
    [0, 0, 0, 1, -1, 0],  # NVDA vs TSLA view
    [0, 0, 0, 0, -1, 1],  # TSLA vs META view
])
q= np.array([0.10, 0.05, 0.0, 0.02, -0.01])

np.random.seed(123)
omega_samples = np.random.uniform(0.0001, 0.01, size=(len(q)))
omega=np.diag(omega_samples)

- The matrix Ω in the Black-Litterman model represents the uncertainty associated with our views. Each diagonal element corresponds to the uncertainty of the respective view. The larger the value on the diagonal, the higher the uncertainty. The values on the diagonal will make the resulting portfolio weights vary.
- We choose to use a random generation of values so we can increase the size and see how the values of Omega make the portfolio weights vary

# Question 4
- Use these views to compute the conditional expected excess return and conditional covariance matrix of excess returns μBL and ΣBL.

In [13]:
t=0.025 # As the value in the course
p_transpose=P.T
t_covmatrix=np.linalg.inv(t*cov_matrix)
matrix_1=np.linalg.inv(t_covmatrix+p_transpose@np.linalg.inv(omega)@P)
matrix_2=t_covmatrix@(data_weights['Implied CAPM excess returns %']/100)+p_transpose@np.linalg.inv(omega)@q
return_bl=matrix_1@matrix_2
data_weights['Returns BL %']=return_bl*100
data_weights

,Mean Excess return %,Weights Market %,Weights MVU %,Implied CAPM excess returns %,Returns BL %
AAPL,2.041488,7.10,-18.505199,0.377696,0.723363
MSFT,2.057845,6.51,-23.694878,0.407228,0.817028
AMZN,0.951350,3.24,-30.376338,0.313083,0.549121
NVDA,2.231979,2.84,150.187995,0.266783,0.506340
TSLA,4.588607,1.87,57.565217,0.498379,0.851437
META,3.798792,1.84,16.323382,0.629454,1.022709


- Now we compute the conditional covariance matrix of excess returns

In [14]:
sigma_bl=cov_matrix+matrix_1
sigma_bl

,AAPL,AMZN,META,MSFT,NVDA,TSLA
AAPL,0.007298,0.004290,0.003273,0.003382,0.006645,0.008314
AMZN,0.004290,0.008617,0.004231,0.004032,0.006640,0.007322
META,0.003273,0.004231,0.009259,0.002857,0.004486,0.004659
MSFT,0.003382,0.004032,0.002857,0.004114,0.005211,0.005020
NVDA,0.006645,0.006640,0.004486,0.005211,0.018377,0.007798
TSLA,0.008314,0.007322,0.004659,0.005020,0.007798,0.033762


# Question 5:
- Use μBL and ΣBL to compute the mean-variance weights and compare them with the weights from the CAPM and the weights based on sample moments.

In [15]:
weight_bl=1/gamma*np.linalg.inv(sigma_bl)@return_bl
data_weights['Weights BL %']=weight_bl*100
data_weights

,Mean Excess return %,Weights Market %,Weights MVU %,Implied CAPM excess returns %,Returns BL %,Weights BL %
AAPL,2.041488,7.10,-18.505199,0.377696,0.723363,17.631097
MSFT,2.057845,6.51,-23.694878,0.407228,0.817028,17.893435
AMZN,0.951350,3.24,-30.376338,0.313083,0.549121,3.160976
NVDA,2.231979,2.84,150.187995,0.266783,0.506340,6.159335
TSLA,4.588607,1.87,57.565217,0.498379,0.851437,-0.261915
META,3.798792,1.84,16.323382,0.629454,1.022709,0.492824
